# Text classification with word embedding

sequences matrix → 直接将所有单词的词向量相加

效果比较差

In [3]:
from src.data_preprocess.preprocess import DataProcessor

data_processor = DataProcessor()
train_documents, test_documents = data_processor.get_train_and_test_documents()

x_train = []
y_train = []
x_test = []

for i, document in enumerate(train_documents):
    for j, label in enumerate(document.class_list):
        x_train.append(document.text)
        y_train.append(label)

for i, document in enumerate(test_documents):
    x_test.append(document.text)

========== Extract data from pickle files ==========


In [4]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

data_tok = [tokenizer.tokenize(d.lower()) for d in x_train]

In [5]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [6]:
import numpy as np
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. Just take an 
    average of vectors for all tokens in the phrase with some weights.
    """
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    # 1. lowercase phrase
    phrase = phrase.lower()
    
    # 2. tokenize phrase
    phrase_tokens = tokenizer.tokenize(phrase)
    
    # 3. average word vectors for all words in tokenized phrase, skip words that are not in model's vocabulary
    divisor = 0
    for word in phrase_tokens:
        if word in model.vocab:
            divisor += 1
            vector = vector + model.get_vector(word)
    
    if divisor != 0: vector /= divisor
    
    return vector

In [7]:
vector_matrix_x_train = list(map(get_phrase_embedding, x_train))

In [8]:
vector_matrix_x_test = list(map(get_phrase_embedding, x_test))

In [9]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from src.metric.metric import cal_predict_accuracy
svm_model = SVC()

svm_model.fit(vector_matrix_x_train, y_train)
y_pred = svm_model.predict(vector_matrix_x_test)

/Users/gjxhlan/anaconda3/envs/AI/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [10]:
print(cal_predict_accuracy(y_pred=y_pred, test_documents=test_documents))

0.5696391291380853
0.5696391291380853
